In [1]:
!apt install proj-bin libproj-dev libgeos-dev
!pip install shapely cartopy 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libgdal-doc
The following NEW packages will be installed:
  libgeos-dev libproj-dev proj-bin
0 upgraded, 3 newly installed, 0 to remove and 8 not upgraded.
Need to get 305 kB of archives.
After this operation, 1,706 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgeos-dev amd64 3.6.2-1build2 [73.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libproj-dev amd64 4.9.3-2 [199 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 proj-bin amd64 4.9.3-2 [32.3 kB]
Fetched 305 kB in 2s (152 kB/s)
Selecting previously unselected package libgeos-dev.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../libgeos-dev_3.6.2-1build2_amd64.deb ...
Unpacking libgeos-dev (3.6.2-1build2) ...
Selecting previously unselected package libproj-dev:amd64.
Preparin

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# 日本語フォントをダウンロードする。
!apt-get -y install fonts-ipafont-gothic

# キャッシュを削除する。
!rm /root/.cache/matplotlib/fontList.json　# 旧cache
!rm /root/.cache/matplotlib/fontlist-v300.json # 消すべきcache

#ここでランタイムを再起動する


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-ipafont-mincho
The following NEW packages will be installed:
  fonts-ipafont-gothic fonts-ipafont-mincho
0 upgraded, 2 newly installed, 0 to remove and 8 not upgraded.
Need to get 8,251 kB of archives.
After this operation, 28.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-ipafont-gothic all 00303-18ubuntu1 [3,526 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-ipafont-mincho all 00303-18ubuntu1 [4,725 kB]
Fetched 8,251 kB in 3s (2,777 kB/s)
Selecting previously unselected package fonts-ipafont-gothic.
(Reading database ... 131232 files and directories currently installed.)
Preparing to unpack .../fonts-ipafont-gothic_00303-18ubuntu1_all.deb ...
Unpacking fonts-ipafont-gothic (00303-18ubuntu1) ...
Selecting previously unselected package fonts-ipa

In [0]:
import numpy as np 
import matplotlib as mpl        
import matplotlib.pyplot as plt 
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.io.img_tiles import GoogleWTS
import six
from PIL import Image
import cartopy.io.shapereader as shpreader
%matplotlib inline

font = {'family':'IPAGothic'}
import seaborn as sns
sns.set(font='IPAGothic')

def new_get_image(self, tile):
    if six.PY3:
        from urllib.request import urlopen, Request
    else:
        from urllib2 import urlopen
    url = self._image_url(tile)  # added by H.C. Winsemius
    req = Request(url) # added by H.C. Winsemius
    req.add_header('User-agent', 'your bot 0.1')
    # fh = urlopen(url)  # removed by H.C. Winsemius
    fh = urlopen(req)
    im_data = six.BytesIO(fh.read())
    fh.close()
    img = Image.open(im_data)

    img = img.convert(self.desired_tile_form)

    return img, self.tileextent(tile), 'lower'

# https://github.com/SciTools/cartopy/issues/1341
class MyMap(GoogleWTS):
    # http://developer.mapquest.com/web/products/open/map for terms of use
    def _image_url(self, tile):
        x, y, z = tile
        url = 'https://cyberjapandata.gsi.go.jp/xyz/std/%s/%s/%s.png' % (z, x, y)
        return url
      
cimgt.GoogleWTS.get_image = new_get_image
request = MyMap()

name = "地理院タイル 標準地図"
filename = "cyberjapan_std.svg"

extent = [138.884354, 139.771500, 35.705065, 35.099687]

fig2 = plt.figure(figsize=(10, 10))

ax = plt.axes(projection=request.crs)
ax.set_extent(extent)

f1 = 'drive/My Drive/Colab Notebooks/data/N03-190101_14_GML-m/N03-190101_14_GML-m.shp'
s1 = list(shpreader.Reader(f1).geometries())
ax.add_geometries(s1, ccrs.PlateCarree(), edgecolor='red', facecolor="gray", linewidth=3, alpha=0.3)

f2 = 'drive/My Drive/Colab Notebooks/data/P20-12_14_GML-u/P20-12_14_GML-u.shp'
points = list(shpreader.Reader(f2).geometries())

#ax.set_title('神奈川県')
ax.set_title(name)
ax.scatter([point.x for point in points],
           [point.y for point in points],
           transform=ccrs.Geodetic() , c='green',s=50, label='避難所', edgecolor='black', linewidth=1)

ax.coastlines(resolution='10m')
ax.legend(loc='upper left')

ax.add_image(request, 10)
plt.show()

fig2.savefig("drive/My Drive/Colab Notebooks/svg/" + filename, dpi=300, facecolor='None', edgecolor='None', transparent=True, format="svg")
